<a href="https://colab.research.google.com/github/Seblao/R-seaux-Neurones/blob/main/Building_a_VGG16_CNN_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from google.colab import drive

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers

# Tasks
In this quest, you will be working on a classic CNN task of classifying cats and dogs images. You can access the training and testing sets [here](https://drive.google.com/drive/folders/19EL9VyYCVqxStG-O1jJtBQGa8JZpLDvO?usp=sharing) and get acquainted with the folder structure.

Make sure to change your Colab runtime to GPU to ensure adequate performance.

Specify 4 directories for cats and dogs images of training and testing sets respectively. You will be using them to build and train the network.

In [ ]:
├── training_set/
│   ├── cats/
│   │   ├── cat001.jpg
│   │   ├── cat002.jpg
│   │   └── ...
│   └── dogs/
│       ├── dog001.jpg
│       ├── dog002.jpg
│       └── ...
├── test_set/
    ├── cats/
    │   ├── cat101.jpg
    │   ├── cat102.jpg
    │   └── ...
    └── dogs/
        ├── dog101.jpg
        ├── dog102.jpg
        └── ...

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
root_path = '/content/drive/MyDrive'
print(os.listdir(root_path))

['WildCodeSchool', 'Colab Notebooks', 'EBP', 'Simplon', 'ACEDM', 'Clinifutur', "Fréquence d'utilisation des variables par processus de contrôle.xlsx", 'EnqueteGlobale_17120224.ipynb', 'Attestation Jérémy PAROT.docx', '20250423_091344[1].jpg']


In [4]:
data_path = os.path.join(root_path, 'WildCodeSchool')
print(os.listdir(data_path))

['Banquebdd.sql', 'Copie de Untitled1.ipynb', 'Challenge.Création.Business.Modèle.docx', 'Copie de Quête Introduction au Machine Learning - Régréssion linéaire simple, multiple et polynomiale.ipynb', 'Copie de Quete_Logistic_Regression_Titanic.ipynb', 'Copie de  1.3 ML - Train test split.ipynb', 'Copie de Copie de Quete Titanic re.ipynb', 'Copie de Standardize your data.ipynb', 'CNN – Utilisat° de mdèles pré-entraînés.ipynb', 'test_set', 'training_set']


In [5]:
train_dir = '/content/drive/MyDrive/WildCodeSchool/training_set'
test_dir = '/content/drive/MyDrive/WildCodeSchool/test_set'

Initiate `ImageDataGenerator` (check the [documentation](https://keras.io/api/preprocessing/image/)). Your image generator needs to convert the image from RGB into number arrays in order for the neural network to proces them (in other words, normalize the values using the `rescale` parameter).

In [6]:
# Préparat° des données avec ImageDataGenerator :
# Générateur pr l'entraînemt (avec nrmalisat° des images 0-1) :
train_datagen = ImageDataGenerator(rescale=1.0/255)

# Générateur pr le test (égalemt nrmalisé) :
test_datagen = ImageDataGenerator(rescale=1.0/255)

Apply the `ImageDataGenerator` you defined above using `flow_from_directory` to convert both training and testing data.

In [9]:
# Chrgemt des images depuis les dossiers :
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224), # Taille d'entrée VGG16
    batch_size=32,
    class_mode='binary'  # binaire = chat ou chien
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Build a convolutional neural network based on the [VGG16](https://neurohive.io/en/popular-networks/vgg16/) architecture.

In [10]:
# Cstruct° du modèle CNN basé sur VGG16 :
# Chrge VGG16 sans la partie classificat° (top) :
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches de VGG16 pr ne pas les réentraîner :
conv_base.trainable = False

# Ajte 1 tête personnalisée :
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binaire (car chat ou chien)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Compile it using the optimizer and error metrics of your choice.

In [11]:
# Cpilat° du mdèle :
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Fit the model using `fit_generator`. This time, feel free to define the parameters according to your understanding and experiment with them to find a better solution. Be patient though, the training will take a while.

In [1]:
# Entraînemt du mdèle :
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

NameError: name 'model' is not defined

Plot the model accuracy.

In [ ]:
# Visualisat° de la précis° :
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(8, 5))
plt.plot(epochs, acc, 'bo-', label='Training acc')
plt.plot(epochs, val_acc, 'ro-', label='Validation acc')
plt.title('Accuracy du modèle')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()